# TrainingServingSkew
* Moniitors whether the alphas generated for inference are identical 
    to the alphas generated during training

In [ ]:
username = ""
task = ""
medium = ""

In [ ]:
source_name = "TrainingServingSkew"

In [ ]:
import NBInclude: @nbinclude
import Statistics: mean
@nbinclude("../InferenceAlphas/Alpha.ipynb");

In [ ]:
function read_training_alpha(
    alpha::String,
    split::String,
    medium::String;
    content = nothing,
)
    uid = parse(Int, get_recommendee_username()) + 1
    if isnothing(content)
        if occursin("ItemCount", alpha) ||
           occursin("implicit", lowercase(alpha)) ||
           alpha in [
               "$medium/all/UserAverage",
               "$medium/all/UserVariance",
               "$medium/all/SimpleAverage",
               "$medium/all/SequelSeries",
               "$medium/$task/MLE.Ensemble",
           ]
            content = "implicit"
        elseif occursin("explicit", lowercase(alpha))
            content = "explicit"
        end
        @assert !isnothing(content) alpha
    end
    df = read_alpha(alpha, split, task, content, medium)
    filter(df, df.user .== uid)
end;

In [ ]:
function get_errors(alpha, split, medium, content, fn)
    serving = read_recommendee_alpha(alpha, "all", medium)
    training = read_training_alpha(alpha, split, medium; content = content)
    if length(training.item) == 0
        return 0
    end
    fn(training, serving)

    abs.(training.rating .- serving.rating[training.item])
end

function average_errors(alpha, split, medium; content = nothing)
    get_errors(
        alpha,
        split,
        medium,
        content,
        (training, serving) -> abs.(training.rating .- serving.rating[training.item]),
    )
end

function relative_errors(alpha, split, medium; content = nothing)
    get_errors(
        alpha,
        split,
        medium,
        content,
        (training, serving) ->
            abs.(
                abs.(training.rating) ./
                (abs.(serving.rating[training.item]) .+ eps(Float64)) .- 1,
            ),
    )
end;

In [ ]:
function maximum_error(alpha, split, medium; content = nothing)
    maximum(average_errors(alpha, split, medium; content = content))
end

function average_abs_error(alpha, split, medium; content = nothing)
    mean(average_errors(alpha, split, medium; content = content))
end

function average_rel_error(alpha, split, medium; content = nothing)
    mean(relative_errors(alpha, split, medium; content = content))
end;

In [ ]:
function display_errors(alphas, split, medium; content = nothing)
    if isempty(alphas)
        return
    end
    alpha_padding = maximum(length.(alphas))
    number_padding = 16

    header_1 = rpad("Alpha", alpha_padding)
    header_2 = rpad("Avg Abs Error", number_padding)
    header_3 = rpad("Avg Rel Error", number_padding)
    header_4 = rpad("Max Error", number_padding)
    @info "$header_1 $header_2 $header_3 $header_4"

    for alpha in alphas
        GC.gc()
        avg_abs_error =
            rpad(average_abs_error(alpha, split, medium; content = content), number_padding)
        avg_rel_error =
            rpad(average_rel_error(alpha, split, medium; content = content), number_padding)
        max_error =
            rpad(maximum_error(alpha, split, medium; content = content), number_padding)
        alpha = rpad(alpha, alpha_padding)
        @info "$alpha $avg_abs_error $avg_rel_error $max_error"
    end
end;

In [ ]:
potential_alphas = vcat(
    [
        ["$medium/$t/" * x for x in readdir(recommendee_alpha_basepath() * "/$medium/$t")] for t in ["all", task]
    ]...,
    ["$medium/$task/Transformer/v1/explicit", "$medium/$task/Transformer/v1/implicit"],
)
alphas = sort([
    x for
    x in potential_alphas if ispath("$(recommendee_alpha_basepath())/$(x)/alpha.jld2")
])
validation_alphas = sort(["$medium/$task/Linear$x" for x in ["Explicit", "Implicit"]])
# test_alphas = sort(["$medium/$task/MLE.Ensemble"])
test_alphas = String[]
inference_only_alphas = sort(
    vcat(
        [
            "$medium/all/RecapSeries",
            "$medium/all/RelatedSeries",
            "$medium/all/CrossRelatedSeries",
            "$medium/all/CrossRecapSeries",
        ],
        [x for x in alphas if occursin("MLE.Ensemble", x)],
    ),
)
training_alphas = [
    x for
    x in alphas if x ∉ validation_alphas && x ∉ test_alphas && x ∉ inference_only_alphas
];

# Regular splits

In [ ]:
display_errors(training_alphas, "training", medium)

In [ ]:
# display_errors(training_alphas, "validation")
# println()
# display_errors(validation_alphas, "validation")

In [ ]:
display_errors(training_alphas, "test", medium)
println()
display_errors(validation_alphas, "test", medium)
println()
display_errors(test_alphas, "test", medium)

# Negative splits

In [ ]:
# display_errors(training_alphas, "training"; content = "negative")

In [ ]:
# display_errors(training_alphas, "validation"; content = "negative")
# println()
# display_errors(validation_alphas, "validation"; content = "negative")

In [ ]:
display_errors(training_alphas, "test", medium; content = "negative")
println()
display_errors(validation_alphas, "test", medium; content = "negative")
println()
display_errors(test_alphas, "test", medium; content = "negative")